In [277]:
#import packages

import numpy as np
import pandas as pd
import random
import pickle #importing file
import itertools
import folium #mapping
import seaborn as sns #color palette
import plotly
import chart_studio.plotly as py #!pip install chart_studi#o
import datetime
from datetime import date
from pyproj import Geod
#import pyplot_themes as themes

In [278]:
import time
timestamp = time.strftime("%Y-%m-%d-%H:%M:%S") #+"/"
timestamp

import os
os.makedirs(timestamp)

In [279]:
# import data

file_root = '../data/results/'
file_stem = 'full_detailed_results' #'results_pooled_cement_only_complexcost'
file_format = '.pkl'
file_name = file_root+file_stem+file_format #'../data/results/results_pooled_cement_only_complexcost.pkl' ##results_toydata_uniform.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [280]:
#convert to dataframe and parse the geolocation

df = pd.DataFrame(loaded_list, columns = ['fac name','supplier', 'pool','receiver', 'distance', 'cost','volume','cost per ton','industry','timeframe'])

#convert suppliers to list to get around weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()

#split the lists of paired geolocations into two elements each, lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lon


#clean names
fac_names = list(df['fac name'])
fac_names_clean = ['Emitter: ' +str(item) for item in fac_names ]

pool_names = list(df['pool'])
pool_names= ["Pool ID: " + str(item).replace('.0','') if '.0' in str(item) else item for item in pool_names]

pool_names


df['fac name clean'] = fac_names_clean
df['pool name clean'] = pool_names

df.head()

,fac name,supplier,pool,receiver,distance,cost,volume,cost per ton,industry,timeframe,supplier_lat,supplier_lon,receiver_lat,receiver_lon,fac name clean,pool name clean
0,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Asphalto_Any Area_Antelope Shale,"[35.282426, -119.56323450000002]",146.754233,2.002706,0.115223,17381.111222,cement,1,34.6222,-117.1001,35.282426,-119.563235,Emitter: CEMEX Construction Materials Pacific LLC,Asphalto_Any Area_Antelope Shale
1,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Bowerbank_Any Area_No Pool Breakdown,"[35.421670000000006, -119.4112865]",141.956006,2.001750,0.077034,25985.234166,cement,1,34.6222,-117.1001,35.421670,-119.411287,Emitter: CEMEX Construction Materials Pacific LLC,Bowerbank_Any Area_No Pool Breakdown
2,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Buena Vista_Buena Vista Hills_Upper (To-Etcheg...,"[35.1898, -119.461314]",139.418375,2.000124,0.005577,358610.543618,cement,1,34.6222,-117.1001,35.189800,-119.461314,Emitter: CEMEX Construction Materials Pacific LLC,Buena Vista_Buena Vista Hills_Upper (To-Etcheg...
3,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Cache Creek Gas_Any Area_Starkey,"[38.735106815, -121.72364445000002]",382.501960,2.008459,0.138225,14530.372332,cement,1,34.6222,-117.1001,38.735107,-121.723644,Emitter: CEMEX Construction Materials Pacific LLC,Cache Creek Gas_Any Area_Starkey
4,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Calders Corner_Any Area_Stevens,"[35.392538, -119.255733]",133.094814,2.000982,0.046123,43383.808295,cement,1,34.6222,-117.1001,35.392538,-119.255733,Emitter: CEMEX Construction Materials Pacific LLC,Calders Corner_Any Area_Stevens


In [281]:
#create a color palette, repeat it 100 times so we don't run out of colors for large graphs
palette = ['red', 'green', 'purple', 'orange',  'darkblue',
           'gray','cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', # 'darkred','lightred' these coolors looked very similar to other colors, commented out
           'black','lightgray','lightred','blue','beige']*100

In [282]:
#for testing
#df = pd.concat([df.tail(10), df.head(10)])
#df

## Sankey Diagram

In [283]:
industries = set(list(df['industry']))
timeframes = set(list(df['timeframe']))

In [284]:

#df = df.sort_values('volume') sorting is done inside the sankey by some complex algo

In [285]:
#functionalize
def plot_sankey(plot_df,industry,timeframe):

    df = plot_df
    cat_cols=['fac name clean','pool name clean']
    value_cols='volume'
    title='sankey_'+file_stem+'_'+industry+'_'+timeframe+'_year'
    industry = industry
    timeframe = timeframe

    colorPalette = palette #['#4B8BBE', '#4B8BBE'] #,'#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []

    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp

    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))

    # # define colors based on number of levels
    # colorList = palette #[]
    # for idx, colorNum in enumerate(colorNumList): #count how many colors you need
    #     print(idx, colorNum)
    #     colorList = colorList + [colorPalette[idx]]*colorNum 

    colorList = []
    ix = 0

    for i in range(0,len(labelList)):
        #if the item contains emitter, increment the color
        if 'Emitter' in labelList[i]: 
            colorList.append(palette[ix])
            ix = ix+1
        else:
            colorList.append('Black')    

    #colorList

    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','sum']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','sum']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'sum':'sum'}).reset_index()

    print (sourceTargetDf.head())

    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))

    # creating the sankey diagram
    data = dict(
        type='sankey',
        valueformat = ".2f",
        valuesuffix = "K tons",
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = 'black' # colorList #
        ),
        link = dict(
          valueformat = ".2f",
          valuesuffix = "K tons",
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['sum']# ['Volume:' + str(item) for item in list(sourceTargetDf['sum'])]
        )
      )

    layout =  dict(
        #title = title,
        font = dict(
          size = 10
        )
    )

    fig = dict(data=[data], layout=layout, font=dict(size=8, color='white'), paper_bgcolor='#51504f')

    sankey_filename = 'sankey_'+file_stem+'_'+industry+'_'+timeframe+'_year.html'
    #print(sankey_filename)
    #plotly.offline.plot(fig, validate=False, filename= timestamp+'//'+sankey_filename)
    plotly.offline.plot(fig, validate=False, filename= timestamp+'//'+sankey_filename,auto_open=False)

In [286]:
#print(industry, timeframe)
plot_df = df[(df['industry'] =='cement') & (df['timeframe'] == 1)]

plot_df = plot_df.sort_values('cost per ton')
plot_df = plot_df.head(20)
        
        #call plot funciton 
plot_sankey(plot_df,'cement', str(1))
plot_df

                                              source  \
0  Emitter: CEMEX Construction Materials Pacific LLC   
1  Emitter: CEMEX Construction Materials Pacific LLC   
2  Emitter: CEMEX Construction Materials Pacific LLC   
3  Emitter: CEMEX Construction Materials Pacific LLC   
4  Emitter: CEMEX Construction Materials Pacific LLC   

                                    target       sum  
0  Cat Canyon_Gato Ridge_No Pool Breakdown  0.572835  
1           Chowchilla Gas_Any Area_Garzas  0.468865  
2     Cymric_Sheep Springs_Phacoides (ABD)  0.671836  
3       Cymric_Welport_McDonald-Devilwater  0.566111  
4    Las Cienegas_Murphy_No Pool Breakdown  0.390645  


,fac name,supplier,pool,receiver,distance,cost,volume,cost per ton,industry,timeframe,supplier_lat,supplier_lon,receiver_lat,receiver_lon,fac name clean,pool name clean
78,LEHIGH SOUTHWEST CEMENT CO.,"[40.7369, -122.3223]",Holser_Any Area_Holser-Nuevo,"[34.43315812, -118.7405843]",477.543526,2.066459,0.869803,2375.779338,cement,1,40.736900,-122.322300,34.433158,-118.740584,Emitter: LEHIGH SOUTHWEST CEMENT CO.,Holser_Any Area_Holser-Nuevo
87,CalPortland Company Mojave Plant,"[35.029298, -118.316236]",Antelope Hills_Williams_Upper,"[35.514488, -119.851762]",92.877513,2.010581,0.712000,2823.849166,cement,1,35.029298,-118.316236,35.514488,-119.851762,Emitter: CalPortland Company Mojave Plant,Antelope Hills_Williams_Upper
68,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Wilmington_Fault Block 90_Ranger,"[33.73989, -118.139243]",85.111898,2.009518,0.698935,2875.113668,cement,1,34.622200,-117.100100,33.739890,-118.139243,Emitter: CEMEX Construction Materials Pacific LLC,Wilmington_Fault Block 90_Ranger
10,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Cymric_Sheep Springs_Phacoides (ABD),"[35.389354, -119.72621200000002]",157.784799,2.016961,0.671836,3002.164282,cement,1,34.622200,-117.100100,35.389354,-119.726212,Emitter: CEMEX Construction Materials Pacific LLC,Cymric_Sheep Springs_Phacoides (ABD)
86,NATIONAL CEMENT CO OF CALIFORNIA INC,"[34.819863, -118.748732]",Wilmington_Fault Block V-A_Upper Terminal,"[33.75834274, -118.2159882]",79.398795,2.007896,0.621567,3230.379348,cement,1,34.819863,-118.748732,33.758343,-118.215988,Emitter: NATIONAL CEMENT CO OF CALIFORNIA INC,Wilmington_Fault Block V-A_Upper Terminal
85,NATIONAL CEMENT CO OF CALIFORNIA INC,"[34.819863, -118.748732]",Sespe_Tar Creek-Topatopa_Basal Sespe,"[34.476167115, -118.88099265]",24.908675,2.002420,0.607219,3297.687796,cement,1,34.819863,-118.748732,34.476167,-118.880993,Emitter: NATIONAL CEMENT CO OF CALIFORNIA INC,Sespe_Tar Creek-Topatopa_Basal Sespe
6,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Cat Canyon_Gato Ridge_No Pool Breakdown,"[34.79319942, -120.2582128]",179.760266,2.016476,0.572835,3520.170551,cement,1,34.622200,-117.100100,34.793199,-120.258213,Emitter: CEMEX Construction Materials Pacific LLC,Cat Canyon_Gato Ridge_No Pool Breakdown
11,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Cymric_Welport_McDonald-Devilwater,"[35.345144, -119.661809]",153.372216,2.013892,0.566111,3557.417051,cement,1,34.622200,-117.100100,35.345144,-119.661809,Emitter: CEMEX Construction Materials Pacific LLC,Cymric_Welport_McDonald-Devilwater
30,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Midway-Sunset_Any Area_Potter,"[35.26512356, -119.574142]",146.989868,2.012840,0.545950,3686.859928,cement,1,34.622200,-117.100100,35.265124,-119.574142,Emitter: CEMEX Construction Materials Pacific LLC,Midway-Sunset_Any Area_Potter
83,NATIONAL CEMENT CO OF CALIFORNIA INC,"[34.819863, -118.748732]",Elk Hills_Any Area_Stevens (Main Body B Sand),"[35.2773985, -119.3897485]",48.104397,2.003848,0.499933,4008.235816,cement,1,34.819863,-118.748732,35.277398,-119.389748,Emitter: NATIONAL CEMENT CO OF CALIFORNIA INC,Elk Hills_Any Area_Stevens (Main Body B Sand)


In [287]:
#sourceTargetDf

In [288]:
#colorList

## Map the Data

In [289]:
#pre-work for the map 

def plot_map(plot_df,industry,timeframe):

    #find unique suppliers since we only need to plot them once
    unique_suppliers = plot_df[['supplier_lat', 'supplier_lon','fac name']]
    unique_suppliers = unique_suppliers.drop_duplicates()
    unique_suppliers =unique_suppliers.reset_index(drop=True)
    unique_suppliers

    #find max and quantile of volume for setting up thicker lines
    volumes = plot_df['volume']
    quantiles = np.quantile(volumes, [0.2,0.4,0.6,0.8])
    max_volume = np.max(volumes)


    #initialize the count of receivers to 0
    receiver_count = 0

    # Create a map centered in CA
    mapit = folium.Map( location=[36.3302, -119.2921], zoom_start=6) 

    #loop through unique suppliers (outer loop)
    #and receivers for each supplier (inner loop) 
    for i in range(0,unique_suppliers.shape[0]):

        #find all receivers associated with the supplier late and lon, filter the dataframe down to receivers
        #for this particular supplier 
        s_lat = unique_suppliers['supplier_lat'][i]
        s_lon = unique_suppliers['supplier_lon'][i]
        s_name = unique_suppliers['fac name'][i]

        #big circle for supplier
        folium.CircleMarker( #RegularPolygonMarker( #CircleMarker( 
                            location=[ s_lat,s_lon ], 
                            #fill=True, 
                            fill_color=palette[i], 
                            color=palette[i],
                            #number_of_sides=6,
                            tooltip = 'Emitter: ' + s_name, #+str(s_lat)+', '+str(s_lon),
                            radius=4
                            ).add_to( mapit ) #'


        receivers = plot_df[(plot_df['supplier_lat'] == s_lat) & (plot_df['supplier_lon'] == s_lon)]

        #plot the receivers with a new color each time
        latlon = receivers['receiver']
        pool_ids = receivers['pool']
        receiver_count += receivers.shape[0] #to print at the end as a check

        #for each receiver  
        for j in range(0,len(latlon)): 

            #split apart into lat and lon
            lat = latlon.iloc[j][0]
            lon = latlon.iloc[j][1]
            pool_id = pool_ids.iloc[j]

            volume = receivers.iloc[j]['volume']

            #line coordinates
            line_coords = [ [s_lat, s_lon], [lat, lon] ]

            line_tooltip = str('Emitter: ')+s_name+ str('<br>Pool ID: ')+ pool_id

            #create line
            my_PolyLine= folium.PolyLine(locations=line_coords,
                                         color=palette[i],
                                         tooltip = line_tooltip,
                                         weight=2) #color and supplier use i index

            #add line to map
            mapit.add_child(my_PolyLine)
            
            geodesic = Geod(ellps='WGS84')
            
            #rotation = [geodesic.inv(pair[0][1], pair[0][0], pair[1][1], pair[1][0])[0]+90 for pair in pairs]
            rot = geodesic.inv(lon, lat, s_lon, s_lat)[0]+90
            
            folium.RegularPolygonMarker(location=[ lat,lon], 
                                        fill_color=palette[i], 
                                        color=palette[i],  
                                        number_of_sides=3, 
                                        radius=4,
                                        tooltip =  'Pool ID: '+ str(pool_id)
                                        , rotation=rot
                                       ).add_to(mapit)

            
#             #add dot for the receiver
#             folium.CircleMarker(location=[ lat,lon ],
#                                 #bounds=[ lat,lon ],
#                                fill=True, 
#                                 fill_color=palette[i], 
#                                 color=palette[i], 
#                                 tooltip =  'Pool ID: '+ str(pool_id),#'receiver tooltip placeholder',
#                                 radius=5 ).add_to( mapit )

#             icon_square = folium.Icon(
#             icon_shape='rectangle-dot', 
#             border_color=palette[i], 
#             border_width=10)
        
#             folium.Marker( location=[ lat,lon ],  tooltip =  'Pool ID: '+ str(pool_id)#'receiver tooltip placeholder',
#                           , icon=icon_square).add_to(mapit)
            

    print("mapped ", receiver_count, "unique receivers")
    mapit #can run this in the notebook if you need to view it there. better to just open the .html in another browser tab

    #save the file
    mapit.save(outfile= timestamp+'//'+'map_'+file_stem+'_'+industry+'_'+str(timeframe)+'_year.html')
    
    mapit


In [290]:
df[(df['industry'] =='cement') & (df['timeframe'] ==1) & (df['fac name'] =='LEHIGH SOUTHWEST CEMENT CO.')]

,fac name,supplier,pool,receiver,distance,cost,volume,cost per ton,industry,timeframe,supplier_lat,supplier_lon,receiver_lat,receiver_lon,fac name clean,pool name clean
78,LEHIGH SOUTHWEST CEMENT CO.,"[40.7369, -122.3223]",Holser_Any Area_Holser-Nuevo,"[34.43315812, -118.7405843]",477.543526,2.066459,0.869803,2375.779338,cement,1,40.7369,-122.3223,34.433158,-118.740584,Emitter: LEHIGH SOUTHWEST CEMENT CO.,Holser_Any Area_Holser-Nuevo
79,LEHIGH SOUTHWEST CEMENT CO.,"[40.7369, -122.3223]",Sespe_Foot of The Hills_Basal Sespe,"[34.46573205, -118.9179582]",471.566094,2.003966,0.052559,38127.885117,cement,1,40.7369,-122.3223,34.465732,-118.917958,Emitter: LEHIGH SOUTHWEST CEMENT CO.,Sespe_Foot of The Hills_Basal Sespe
80,LEHIGH SOUTHWEST CEMENT CO.,"[40.7369, -122.3223]","Shafter, North_Any Area_San Joaquin/Etchegoin","[35.538286, -119.270812]",395.551354,2.009483,0.149838,13411.004159,cement,1,40.7369,-122.3223,35.538286,-119.270812,Emitter: LEHIGH SOUTHWEST CEMENT CO.,"Shafter, North_Any Area_San Joaquin/Etchegoin"


In [291]:
def plot_html(plot_df, industry, timeframe):
    plot_html = plot_df[['fac name','pool','distance','cost','volume', 'cost per ton']].round(decimals=2) 
    #plot_html = plot_df[['fac name','pool','distance','cost','volume', 'cost per ton']].round(decimals=0)   
    plot_html['cost'] = plot_html['cost'].apply(lambda x: "${:.2f}".format((x)))
    #plot_html['volume'] = plot_html['volume'].apply(lambda x: int(x))
    plot_html['distance'] = plot_html['distance'].apply(lambda x: int(x))
    plot_html['cost per ton'] = plot_html['cost per ton'].apply(lambda x: "${:.0f}".format((x)))
    plot_html['pool'] = plot_html['pool'].apply(lambda x: x.replace('_',' '))
    plot_html.columns = ['Carbon Dioxide Emitter','Storage Pool Location','Distance (mi)','Cost (Millions)','Volume (Thousands of Tons)', 'Cost per Ton']
    
    filename=  timestamp+'//'+'html_'+file_stem+'_'+industry+'_'+str(timeframe)+'_year.html'
    
    with open(filename, 'w') as fo:
        fo.write(plot_html.to_html(index=False, justify = 'left', bold_rows = 'True'))

In [292]:
#sankey diagram of each

for industry in industries:
    for timeframe in timeframes:
        #print(industry, timeframe)
        plot_df = df[(df['industry'] ==industry) & (df['timeframe'] == timeframe)]

        plot_df = plot_df.sort_values('cost per ton')
        plot_df = plot_df.head(20)
        
        #call plot funciton
        plot_sankey(plot_df,industry, str(timeframe))
        plot_map(plot_df,industry, str(timeframe))
        plot_html(plot_df,industry, str(timeframe))

                                     source  \
0  Emitter: CHEVRON PRODS.CO. RICHMOND REFY   
1          Emitter: KERN OIL & REFINING CO.   
2          Emitter: KERN OIL & REFINING CO.   
3           Emitter: LUNDAY-THAGARD COMPANY   
4           Emitter: LUNDAY-THAGARD COMPANY   

                                              target         sum  
0                          Stone Lake Gas (ABD)_pool  147.694700  
1                  Aliso Canyon_Any Area_Aliso, West  402.145380  
2              Kern River_Any Area_No Pool Breakdown   70.715276  
3  Coles Levee, North_Any Area_Stevens (Undiffere...  400.917480  
4                                Sansinena_East_pool  428.088920  
mapped  20 unique receivers
                                              source  \
0                    Emitter: Greka Refining Company   
1                    Emitter: Greka Refining Company   
2                   Emitter: KERN OIL & REFINING CO.   
3                Emitter: Los Angeles Refinery (LAR)   
4  Emit

                                              source  \
0  Emitter: CEMEX Construction Materials Pacific LLC   
1  Emitter: CEMEX Construction Materials Pacific LLC   
2  Emitter: CEMEX Construction Materials Pacific LLC   
3  Emitter: CEMEX Construction Materials Pacific LLC   
4  Emitter: CEMEX Construction Materials Pacific LLC   

                                     target       sum  
0    Afton, South, Gas (ABD)_Any Area_Kione  2.594025  
1                       Belridge North_pool  2.672651  
2           Belridge, North_Any Area_Tulare  2.052289  
3  Castaic Junction (ABD)_Any Area_21 North  2.332187  
4                Cat Canyon_Central_Sisquoc  2.066219  
mapped  20 unique receivers
                                              source  \
0  Emitter: CEMEX Construction Materials Pacific LLC   
1  Emitter: CEMEX Construction Materials Pacific LLC   
2  Emitter: CEMEX Construction Materials Pacific LLC   
3  Emitter: CEMEX Construction Materials Pacific LLC   
4  Emitter: CEMEX C

## Map all receivers and emitters

In [293]:

def plot_map_all(plot_df):

    #find unique suppliers since we only need to plot them once
    unique_suppliers = plot_df[['supplier_lat', 'supplier_lon','fac name']]
    unique_suppliers = unique_suppliers.drop_duplicates()
    unique_suppliers =unique_suppliers.reset_index(drop=True)
    unique_suppliers

    #find max and quantile of volume for setting up thicker lines
    volumes = plot_df['volume']
    quantiles = np.quantile(volumes, [0.2,0.4,0.6,0.8])
    max_volume = np.max(volumes)


    #initialize the count of receivers to 0
    receiver_count = 0

    # Create a map centered in CA
    mapit = folium.Map( location=[36.3302, -119.2921], zoom_start=6) 

    #loop through unique suppliers (outer loop)
    #and receivers for each supplier (inner loop) 
    for i in range(0,unique_suppliers.shape[0]):

        #find all receivers associated with the supplier late and lon, filter the dataframe down to receivers
        #for this particular supplier 
        s_lat = unique_suppliers['supplier_lat'][i]
        s_lon = unique_suppliers['supplier_lon'][i]
        s_name = unique_suppliers['fac name'][i]

        #big circle for supplier
        folium.CircleMarker( 
                            location=[ s_lat,s_lon ], 
                            fill=True, 
                            fill_color='blue', 
                            color='blue',
                            tooltip = 'Emitter: ' + s_name, #+str(s_lat)+', '+str(s_lon),
                            radius=1
                            ).add_to( mapit ) #'


        receivers = plot_df[(plot_df['supplier_lat'] == s_lat) & (plot_df['supplier_lon'] == s_lon)]

        #plot the receivers with a new color each time
        latlon = receivers['receiver']
        pool_ids = receivers['pool']
        receiver_count += receivers.shape[0] #to print at the end as a check

        #for each receiver  
        for j in range(0,len(latlon)): 

            #split apart into lat and lon
            lat = latlon.iloc[j][0]
            lon = latlon.iloc[j][1]
            pool_id = pool_ids.iloc[j]

            volume = receivers.iloc[j]['volume']

            #line coordinates
            #line_coords = [ [s_lat, s_lon], [lat, lon] ]

            #line_tooltip = str('Emitter: ')+s_name+ str('<br>Pool ID: ')+ pool_id

            #create line
            #my_PolyLine= folium.PolyLine(locations=line_coords,
             #                            color=palette[i],
             #                            tooltip = line_tooltip,
              #                           weight=2) #color and supplier use i index

            #add line to map
            #mapit.add_child(my_PolyLine)
            
            #add dot for the receiver
            folium.CircleMarker(location=[ lat,lon ],
                                #bounds=[ lat,lon ],
                               fill=True, 
                                fill_color='black', 
                                color='black', 
                                tooltip =  'Pool ID: '+ str(pool_id),#'receiver tooltip placeholder',
                                radius=1 ).add_to( mapit )
            
           
#             icon_square = folium.Icon(
#             icon_shape='rectangle-dot', 
#             border_color=palette[i], 
#             border_width=10)
        
#             folium.Marker( location=[ lat,lon ],  tooltip =  'Pool ID: '+ str(pool_id)#'receiver tooltip placeholder',
#                           , icon=icon_square).add_to(mapit)
            

    print("mapped ", receiver_count, "unique receivers")
    mapit #can run this in the notebook if you need to view it there. better to just open the .html in another browser tab

    #save the file
    mapit.save(outfile= timestamp+'//'+'map_'+file_stem+'_'+'all'+'_'+'all_year.html')
    
    mapit

In [294]:
plot_map_all(df)

mapped  13050 unique receivers


In [295]:
df.head(0)

,fac name,supplier,pool,receiver,distance,cost,volume,cost per ton,industry,timeframe,supplier_lat,supplier_lon,receiver_lat,receiver_lon,fac name clean,pool name clean


In [296]:
plot_html(df.head(0),'all','all')